# Theoretical Headway

In [2]:
import pandas as pd
import numpy as np

In [ ]:
calendar_dates = pd.read_csv('data/timetables/gtfs3Sept/calendar_dates.txt', sep = ",")
calendar = pd.read_csv('data/timetables/gtfs3Sept/calendar.txt', sep = ",")
trips = pd.read_csv('data/timetables/gtfs3Sept/trips.txt', sep = ",")
stops = pd.read_csv('data/timetables/gtfs3Sept/stops.txt', sep = ",")
stop_times = pd.read_csv('data/timetables/gtfs3Sept/stop_times.txt', sep = ",")
routes = pd.read_csv('data/timetables/gtfs3Sept/routes.txt', sep = ",")

## Isolate case computation

In [ ]:
stop = stops.head(1).to_dict(orient="records")[0]
stop

In [ ]:
stop_times_stop = stop_times.merge(
    stops,
    "left",
    "stop_id"
)

In [ ]:
stop_times_stop = stop_times_stop[stop_times_stop.stop_id == stop.get("stop_id")]
stop_times_stop

In [ ]:
stop_times_with_trips = stop_times_stop.merge(
    trips,
    "left",
    "trip_id"
)
route_id = stop_times_with_trips.head(1).to_dict("records")[0].get("route_id")

In [ ]:
stop_times_with_trips = stop_times_with_trips[stop_times_with_trips.route_id == route_id]
stop_times_with_trips

In [ ]:
# Fix direction
direction_id = stop_times_with_trips.head(1).to_dict("records")[0].get("direction_id")

# Fix Service
service_id = stop_times_with_trips.head(1).to_dict("records")[0].get("service_id")

print(direction_id, service_id)

In [ ]:
stop_times_with_trips_stops = stop_times_with_trips[(stop_times_with_trips.direction_id == direction_id) & (stop_times_with_trips.service_id == service_id)]

In [ ]:
time = stop_times_with_trips_stops["arrival_time"].str.split(':', expand = True)
t = time[0].astype(int) * 3600 + time[1].astype(int) * 60 + time[2].astype(int)
stop_times_with_trips_stops["time_seconds"] = t

In [ ]:
stop_times_with_trips_stops = stop_times_with_trips_stops.sort_values(by=["time_seconds"], ascending=True)

In [ ]:
stop_times_with_trips_stops["headway"] = (stop_times_with_trips_stops.time_seconds.sort_values() - stop_times_with_trips_stops.time_seconds.sort_values().shift()) / 60

In [ ]:
stop_times_with_trips_stops.sort_values(by="time_seconds", ascending= True)[["trip_id", "arrival_time","headway", "stop_name", "route_id", "service_id", "trip_headsign"]]

In [ ]:
import plotly.express as px
fig = px.scatter(stop_times_with_trips_stops.sort_values("time_seconds"), x="arrival_time", y="headway", color='headway',  marginal_y="histogram", text="headway")
fig.show()

## All headways Computation

In [ ]:
from custom_functions.gtfs_methods import headways

In [ ]:
stops.shape

In [ ]:
stop_times.shape

In [ ]:
trips.shape

In [ ]:
routes.shape

In [ ]:
trips_with_routes = trips.merge(
    routes,
    "left",
    "route_id"
)
stop_times_stop = stop_times.merge(
    stops,
    "left",
    "stop_id"
)
stop_times_with_trips = stop_times_stop.merge(
    trips_with_routes,
    "left",
    "trip_id"
)
stop_times_with_trips.info(verbose=True)

In [ ]:
headway_theoretical = headways(
    table_times_trips = stop_times_with_trips,
    time_col = "arrival_time",
    stop_col = "stop_id",
    route_col = "route_id",
    direction_col = "direction_id",
    service_col = "service_id"
)

In [3]:
headway_theoretical

NameError: name 'headway_theoretical' is not defined

In [ ]:
headway_theoretical.to_pickle(path = "data/computed/theoretical_schedule.pkl")

In [4]:
headway_theoretical = pd.read_pickle("data/computed/theoretical_schedule.pkl")

FileNotFoundError: [Errno 2] No such file or directory: 'data/computed/theoretical_schedule.pkl'

In [ ]:
t = headway_theoretical[
    (headway_theoretical.stop_id == "0089") & 
    (headway_theoretical.route_id == 40) & 
    (headway_theoretical.direction_id == 0) & 
    (headway_theoretical.service_id == 233817050)]

In [ ]:
import plotly.express as px
fig = px.scatter(t.sort_values("time_seconds"), x="arrival_time", y="headway_min", color='headway_min', text="headway_min")
fig.show()

In [ ]:
k2 = ('9996', 59, 0, 238006605)

t2 = headway_theoretical[
    (headway_theoretical.stop_id == "9996") & 
    (headway_theoretical.route_id == 59) & 
    (headway_theoretical.direction_id == 0) & 
    (headway_theoretical.service_id == 238006605)]

In [ ]:
t2

In [ ]:
fig = px.scatter(t2.sort_values("time_seconds"), x="arrival_time", y="headway_min", color='headway_min', text="headway_min")
fig.show()

In [ ]:
k3 = ('1083', 74, 1, 237569501)

t3 = headway_theoretical[
    (headway_theoretical.stop_id == "1083") & 
    (headway_theoretical.route_id == 74) & 
    (headway_theoretical.direction_id == 1) & 
    (headway_theoretical.service_id == 237569501)]

In [ ]:
t3

In [ ]:
fig = px.scatter(t3.sort_values("time_seconds"), x="arrival_time", y="headway_min", color='headway_min', text="headway_min")
fig.show()

## Service Issue

In [ ]:
calendar_dates

In [ ]:
calendar_dates[calendar_dates.service_id == 238006605]

In [ ]:
calendar[calendar.service_id == 238006605]

In [ ]:
calendar["start_date"] = pd.to_datetime(calendar.start_date, format = "%Y%m%d")
calendar["end_date"] = pd.to_datetime(calendar.end_date, format = "%Y%m%d")

In [ ]:
calendar["diff"] = (calendar["end_date"] - calendar["start_date"]).dt.days

In [ ]:
fig = px.histogram(calendar, x="diff")
fig.show()